<a href="https://colab.research.google.com/github/heewonLEE2/Data-Ai-Colab/blob/main/NLP/Transformer%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PyTorch를 이용해 Transformer 직접 구현해보기**

## 1. Scaled Dot-Product Attention 만들기

In [13]:
import torch
import torch.nn.functional as F
import math

def scaled_dot_product_attention(query, key, value, mask=None):
    # 1. MatMul: Q * K^T
    # query와 key의 차원 중 마지막 차원(d_k)을 기준으로 곱합니다.
    # .transpose(-2, -1)은 마지막 두 차원을 뒤집는 것(전치)입니다.
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1))

    # 2. Scale: 루트 d_k로 나눔
    scores = scores / math.sqrt(d_k)

    # 3. Mask (여기가 핵심!)
    if mask is not None:
        # mask가 0인 부분(가려야 할 부분)을 -1e9로 채웁니다.
        # -inf 대신 -1e9를 쓰는 이유는 안정성 때문입니다.
        scores = scores.masked_fil(mask == 0, -1e9)

    # 4. Softmax
    # dim=-1: 마지막 차원(단어들)에 대해 확률을 구함
    attention_weights = F.softmax(scores, dim=-1)

    # 5. MatMul: 확률 * V
    output = torch.matmul(attention_weights, value)

    return output, attention_weights

## 2. Multi-Head Attention 구현 (PyTorch)

Shape 변환의 마술: "자르고(Split) 돌리기(Swap)"
- 우리의 목표는 [32, 10, 512] 라는 덩어리를 8개의 독립적인 머리가 처리하게 만드는 것입니다.

Step 1. 쪼개기 (View/Reshape) 먼저 마지막 512를 8(개수) x 64(크기)로 논리적으로 나눕니다.
- 변경 전: (Batch, Seq_Len, d_model) -> (32, 10, 512)
- 변경 후: (Batch, Seq_Len, n_head, d_k) -> (32, 10, **8, 64**)

Step 2. 축 돌리기 (Transpose/Swap) - 여기가 핵심! 우리는 8개의 Head가 서로 섞이지 않고 병렬로 계산되길 원합니다. PyTorch의 행렬 곱(matmul)은 마지막 2개 차원만 계산하고, 앞의 차원들은 '배치(Batch)'처럼 취급합니다. 그래서 Head의 축(8)을 문장 길이 축(10)보다 앞으로 끄집어내야 합니다.

- 변경 후: (Batch, **n_head**, Seq_Len, d_k) -> (32, **8**, 10, 64)

> 최종적으로 Attention 연산에 들어가는 Shape은 (32, 8, 10, 64) 가 됩니다. 이렇게 하면 컴퓨터는 마치 "256개(32x8)의 작은 배치"를 처리하는 것처럼 8개의 Head를 동시에 돌립니다.

In [14]:
import torch.nn as nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super().__init__()
        self.n_head = n_head
        self.d_k = d_model // n_head # 512 / 8 = 64

        # 1. W_q, W_k, W_v를 위한 선형층
        # (참고: 각각 만드는 것보다 큰 거 하나 만들어서 쪼개는 게 효율적입니다)\
        self.w_q == nn.Linear(d_model, d_model)
        self.w_k == nn.Linear(d_model, d_model)
        self.w_v == nn.Linear(d_model, d_model)

        # 마지막에 8개 결과를 합치고(Concat) 내보낼 선형층
        self.w_o = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        # q, k, v shape: (batch_size, seq_len, d_model) -> (32, 10, 512)
        batch_size = q.size(0)

        # -------------------------------------------------------
        # 1. Linear & Split & Transpose (핵심!)
        # -------------------------------------------------------
        # view: (32, 10, 8, 64)로 쪼갬
        # transpose: (32, 8, 10, 64)로 축을 바꿈 (1번째와 2번째 차원 교환)
        q = self.w_q(q).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        k = self.w_q(k).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)
        v = self.w_q(v).view(batch_size, -1, self.n_head, self.d_k).transpose(1, 2)

        # -------------------------------------------------------
        # 2. Scaled Dot-Product Attention (아까 만든 함수 사용)
        # -------------------------------------------------------
        # out shape: (32, 8, 10, 64) -> Head별 결과가 나옴
        out, attention_weights = scaled_dot_product_attention(q, k, v, mask)

        # -------------------------------------------------------
        # 3. Concat (다시 붙이기)
        # -------------------------------------------------------
        # transpose: (32, 10, 8, 64)로 다시 원상복구 (Head를 뒤로 보냄)
        # contiguous: 메모리 정렬 (view를 쓰기 위해 필요)
        # view: (32, 10, 512)로 8개 Head를 하나로 합침
        out = out.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)

        # 4. Final Linear
        return self.w_o(out)


### 1. .view()의 -1은 무슨 뜻일까?
한마디로 정의하면 "남은 거 다 너(컴퓨터)가 알아서 계산해!" (Auto-inference) 라는 뜻입니다.

행렬의 모양(Shape)을 바꿀 때, 전체 원소의 개수(Total Elements)는 변하지 않아야 합니다. 만약 전체 원소가 12개인 텐서가 있다고 해봅시다.

- view(3, 4): 3 $\times$ 4 = 12 (OK)
- view(2, 6): 2 $\times$ 6 = 12 (OK)
- view(3, **-1**): 컴퓨터야, 앞이 3이니까 뒤는 뭐가 돼야 12가 나오겠니? -> 4로 자동 변환!

코드에서의 의미

```
q = self.w_q(q).view(batch_size, -1, self.n_head, self.d_k)
```
여기서 원래 텐서의 크기는 (Batch_Size, Seq_Len, d_model)입니다.

1. batch_size: 고정 (예: 32)
2. n_head: 고정 (예: 8)
3. d_k: 고정 (예: 64)
4. -1의 자리: Seq_Len (문장 길이)

> 왜 -1을 썼을까요? 문장의 길이는 매번 다를 수 있기 때문입니다. 10단어짜리 문장이 들어오면 -1은 자동으로 10이 되고, 100단어짜리가 들어오면 100이 됩니다. 길이를 일일이 변수에 담아주지 않아도 돼서 코드가 훨씬 유연해집니다.

## 3. Add & Norm 구현하기

In [15]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_head):
        super.__init__()
        # 1. 아까 만든 멀티 헤드 어텐션
        self.mha = MultiHeadAttention(d_model, n_head)

        # 2. 정규화 층 (Layer Normalization)
        self.norm1 = nn.LayerNorm(d_model)

        # 3. 드롭아웃 (과적합 방지)
        self.dropout1 = nn.Dropout(0.1)

    def forward(self, x, mask):
        # 1. 잔차 연결을 위해 원래 값 저장 (Residual)
        residual = x

        # 2. Attention 수행
        # MHA 내부에서 쪼개고(view) 돌리고(transpose) 다 합니다.
        x = self.mha(x, x, x, mask)

        # 3. 드롭아웃 & Add & Norm
        x = self.dropout1(x)
        x = self.norm1(x + residual)# 여기가 핵심! (원래 값 더하기)

        return x

## 4. FFN 구현 (PyTorch)

논문에서는 이 부분을 Position-wise Feed-Forward Networks라고 부르는데, 이름이 거창하지만 사실 구조는 아주 단순합니다.

1. 확장: 입력을 뻥튀기합니다. (보통 4배, $512 \rightarrow 2048$)
2. 활성화: ReLU로 비선형성을 줍니다. (음수는 자르고 양수만 통과)
3. 압축: 다시 원래 크기로 줄입니다. ($2048 \rightarrow 512$)

> 개발자 관점에서는 "병목(Bottleneck) 구조의 반대"라고 보시면 됩니다. 정보를 넓게 펼쳐서 특징을 더 풍부하게 해석한 뒤, 다시 요약하는 과정입니다.

수식 ($\text{FFN}(x) = \max(0, xW_1 + b_1)W_2 + b_2$)

In [16]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super().__init__()
        # 1. 확장 (512 -> 2048)
        self.w_1 = nn.Linear(d_model, d_ff)
        # 2. 압축 (2048 -> 512)
        self.w_2 = nn.Linear(d_ff, d_model)
        # 3. 활성화 함수
        self.relu = nn.ReLU()
        # 4. 드롭아웃 (옵션)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # Linear -> ReLU -> Dropout -> Linear
        return self.w_2(self.dropout(self.relu(self.w_1(x))))

## 5. Encoder Layer 완성 (조립하기)
자, 이제 Attention과 FFN이라는 두 개의 서브 모듈을 다 만들었습니다. 이걸 하나로 합치면 Transformer의 인코더 한 층(Layer)이 완성됩니다.

아까 작성했던 EncoderLayer 코드에 FFN을 추가해 봅시다. 구조는 데칼코마니처럼 똑같습니다. (Add & Norm이 또 들어갑니다.)

In [17]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff):
        super().__init__()
        # --- [Part 1] Attention ---
        self.mha = MultiHeadAttention(d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model)
        self.dropout1 = nn.Dropout(0.1)

        # --- [Part 2] FFN (새로 추가된 부분) ---
        self.ffn = PositionwiseFeedForward(d_model, d_ff)
        self.norm2 = nn.LayerNorm(d_model) # 두 번째 정규화
        self.dropout2 = nn.Dropout(0.1)

    def forward(self, x, mask):
        # 1. Attention + Add & Norm
        residual = x
        x = self.mha(x, x, x, mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual)

        # 2. FFN + Add & Norm (여기도 똑같은 패턴!)
        residual = x
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual)

        return x

## 6. 전체 Encoder 조립하기

일반 파이썬 list와 nn.ModuleList의 결정적인 차이는 "등록(Registration)" 여부입니다.

- 일반 list: 모델 안에 변수로 가지고는 있지만, PyTorch 엔진(Optimizer)은 이 안에 층이 들어있는지 모릅니다. 그래서 학습을 시켜도 가중치가 업데이트되지 않습니다. (투명 인간 취급)

- nn.ModuleList: "이 리스트 안에 있는 애들은 우리 모델의 부속품이야!"라고 공식적으로 등록해 줍니다. 그래야 나중에 model.parameters()를 호출했을 때 잡힙니다.

지금까지 만든 부품들을 모두 모아서 Encoder 전체 클래스를 완성해 보겠습니다.

1. 임베딩(Embedding): 단어를 벡터로 변환
2. 레이어 쌓기: nn.ModuleList 사용 ($N=6$)
3. 포워딩: for 문을 돌면서 데이터를 통과시킴

In [18]:
class Encoder(nn.Module):
    def __init__(self, input_dim, d_model, n_head, d_ff, n_layers, max_len=5000):
        super().__init__()
        self.d_model = d_model

        # 1. 입력 임베딩 (단어 -> 벡터)
        self.embedding = nn.Embedding(input_dim, d_model)

        # 2. 인코더 레이어 N개 쌓기
        self.layers = nn.ModuleList([
            EncoderLayer(d_model, n_head, d_ff) for _ in range(n_layers)
        ])

        # 3. 드롭아웃
        self.dropout = nn.Dropout(0.1)

    def forward(self, x, mask):
        # 1. 임베딩 + (Positional Encoding은 여기서 더한다고 가정)
        # 보통 x * math.sqrt(self.d_model) 스케일링도 해줍니다.
        x = self.embedding(x)
        x = self.dropout(x)

        # 2. 레이어 통과 (순차적으로)
        for layer in self.layers:
            x = layer(x, mask)

        return x

## 7. Decoder Look-ahead Mask 구현 (PyTorch)

PyTorch의 torch.triu (Triangle Upper) 함수를 이용하면 한 줄로 만들 수 있습니다.

In [19]:
import torch
import numpy as np

def make_subsequent_mask(size):
    "자기 자신과 과거만 볼 수 있게 마스킹 생성"
    attn_shape = (1, size, size)

    # triu: 상부 삼각형(대각선 위쪽)을 1로 만듦 -> 즉, 가려야 할 부분
    # diagonal=1: 대각선(자기 자신)은 제외하고 그 윗칸부터 가림
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')

    # 0인 곳(과거)은 True, 1인 곳(미래)은 False로 리턴하거나
    # 모델 내부 로직에 맞춰 0과 1을 반전시켜 사용
    return torch.from_numpy(subsequent_mask) == 0

## 8. Decoder Layer 조립 (가장 복잡한 부분)

In [20]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, n_head, d_ff):
        super().__init__()
        # 1. 자기들끼리 보는 어텐션 (Masked)
        self.self_attn = MultiHeadAttention(d_model, n_head)
        self.norm1 = nn.LayerNorm(d_model)

        # 2. 인코더 정보를 가져오는 어텐션 (Cross Attention)
        self.src_attn = MultiHeadAttention(d_model, n_head)
        self.norm2 = nn.LayerNorm(d_model)

        # 3. FFN
        self.ffn = PositionwiseFeedForward(d_model, d_ff)
        self.norm3 = nn.LayerNorm(d_model) # 디코더는 정규화가 3개!

        self.dropout = nn.Dropout(0.1)

    def forward(self, x, memory, src_mask, tgt_mask):
        # x: 디코더 입력, memory: 인코더 출력값

        # 1. Masked Self-Attention
        m = memory # 인코더 출력 (K, V로 쓰임)

        # query, key, value 모두 x (자기 자신)
        # tgt_mask: 아까 만든 삼각형 마스크 적용
        x2 = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(x2))

        # 2. Encoder-Decoder Attention (중요!)
        # Query: 디코더의 현재 상태 (x)
        # Key, Value: 인코더의 기억 (memory)
        # src_mask: 인코더 쪽 패딩 가리기용
        x2 = self.src_attn(x, m, m, src_mask)
        x = self.norm2(x + self.dropout(x2))

        # 3. FFN
        x2 = self.ffn(x)
        x = self.norm3(x + self.dropout(x2))

        return x

## 9. Generator (출력층) 구현

PyTorch의 nn.Linear는 아주 똑똑해서, 입력이 3차원(A, B, C)이라도 마지막 차원(C)에만 행렬 곱을 수행합니다. 즉, 문장 길이(Seq_Len) 축은 건드리지 않고 특징만 확장해 줍니다.

In [21]:
class Generator(nn.Module):
    "디코더의 출력을 받아 단어장(vocab) 확률로 변환"
    def __init__(self, d_model, vocab_size):
        super().__init__()
        # 512차원 -> 30,000차원 (단어 개수만큼 뻥튀기)
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        # x shape: (batch, seq_len, d_model)

        # 1. Linear Projection
        # 결과 shape: (batch, seq_len, vocab_size)
        logits = self.proj(x)

        # 2. Softmax
        # dim=-1: 마지막 차원(단어 후보들) 중에서 확률 계산
        return F.log_softmax(logits, dim=-1)

## 최종 조립: Transformer 클래스

> 구조: [Encoder] + [Decoder] + [Src/Tgt Embed] + [Generator]

In [22]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def encode(self, src, src_mask):
        # 소스 문장 임베딩 -> 인코더 통과
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        # 타겟 문장 임베딩 -> 디코더 통과 (인코더의 memory 참조)
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

    def forward(self, src, tgt, src_mask, tgt_mask):
        # 1. 인코더가 소스 문장을 압축(Memory)
        memory = self.encode(src, src_mask)

        # 2. 디코더가 압축된 정보와 현재까지의 타겟을 보고 결과 생성
        res = self.decode(memory, src_mask, tgt, tgt_mask)

        # 3. 단어 확률로 변환 (Generator)
        return self.generator(res)